<a href="https://colab.research.google.com/github/hthomas229/PurpleCrown/blob/main/pssst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Python secrets vs random — Generating True Secure Tokens

When you need a password reset link, an API key, or a one-time token — using **random**() could expose your app to hackers. In this tutorial, we’ll learn why, and how to fix it using Python’s **secrets** module — the right way to generate secure tokens. I’ll also show how Node’s crypto.randomBytes() does the same thing.



```
❌ token = random.random() -> predictable
✅ token = secrets.token_urlsafe() -> cryptographically secure
```



##The Problem with random



1.   The **random** module in Python uses the Mersenne Twister algorithm — great    for simulations, not security.
2.   If someone knows your random seed, they can reproduce every output.
3.   Attackers can predict tokens, session IDs, or password reset links.




In [12]:
import random

random.seed(31)
for _ in range(3):
  print(random.random())

0.01227824739797545
0.11239886108023578
0.39283736155074256


##Use secrets Instead

In [13]:
import secrets

In [42]:
secrets.randbelow(10)  # genreates a random positive integer (or zero) belew (n)

5

Great for dice rolling, random indexing etc.

In [43]:
secrets.token_bytes(16)

b'M@\xf2\x84Ix\xb1NA\xf3\x93\x8f\x8ev\x1b\x83'

Best for binary data for crytographic keys.

In [44]:
secrets.token_hex(16)

'169fed2c9978b83eaaa6f6785b7fb713'

Readable representation

In [45]:
secrets.token_urlsafe(16)

'mWRX1vZVdUvs4XCyn6HR1Q'

Great for password reset links

##Why It’s Secure



*   Backed by cryptographically strong random number generators (CSPRNGs)
*   Even if an attacker sees many tokens, they can’t infer future ones.
*   Uses OS entropy: random values collected from system state (mouse movements, timing noise, etc.).
*   Extremely unlikely to repeat: 2ⁿ possible combinations.

*Imagine rollling 10 ^480 power different dice -- that is the rough equivalent of a 32-bit token*



###Use Case:  Password Reset Token



In [47]:
import secrets
import hashlib
import time

# simulate user requests password reset
user_email = "user@example.com"

# generate secure random token
token = secrets.token_urlsafe(32)
print("Send this link to user:")
print(f"https://myapp.com/reset?token={token}")

# store hashed token (for validation later)
token_hash = hashlib.sha256(token.encode()).hexdigest()
print("Store in DB:", token_hash)

#  set token expiration for 15 minutes
expiry = time.time() + 15 * 60
print("Expiry timestamp:", expiry)


Send this link to user:
https://myapp.com/reset?token=8mmuADuPoY6z8qB8V4vL5__khZGO128WPzJIUDxHNMA
Store in DB: de968d86f1e4598a63edf9e07a74544d894326f49026ed4b762505869c4b121e
Expiry timestamp: 1762378665.8291218




*   Never store tokens in plain text.
*   Hash them to prevent misuse if the database leaks.
*   When user clicks the link, hash their token again and compare.



###Node.js Equivalent  crypto.randomBytes()

Uses system CSPRNG just like Python’s secrets.

base64url makes it safe for URLs.



```
const crypto = require("crypto");

const token = crypto.randomBytes(32).toString("base64url");
console.log("Reset token:", token);

```

